In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import logging
plt.style.use('fivethirtyeight')

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Flatten, Embedding, LSTM, GRU
from tensorflow.keras.models import Sequential

In [4]:
pip install transformers

     |████████████████████████████████| 3.1 MB 7.7 MB/s 
     |████████████████████████████████| 61 kB 558 kB/s 
     |████████████████████████████████| 895 kB 67.8 MB/s 
     |████████████████████████████████| 3.3 MB 41.8 MB/s 
     |████████████████████████████████| 596 kB 53.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [25]:
from transformers import BertTokenizer, BertModel

In [11]:
from nltk.tokenize import word_tokenize

In [ ]:
import json

In [10]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
df1 = pd.read_csv("train_list.csv")

In [6]:
df1.head()

,Unnamed: 0,image,question,answers,answer_type,answerable
0,0,VizWiz_train_00000001.jpg,Can you tell me what is in this can please?,"[{'answer_confidence': 'yes', 'answer': 'soda'...",other,1
1,1,VizWiz_train_00000002.jpg,Is this enchilada sauce or is this tomatoes? ...,"[{'answer_confidence': 'yes', 'answer': 'these...",other,1
2,2,VizWiz_train_00000003.jpg,What is the captcha on this screenshot?,"[{'answer_confidence': 'yes', 'answer': 't36m'...",other,1
3,3,VizWiz_train_00000004.jpg,What is this item?,"[{'answer_confidence': 'yes', 'answer': 'solar...",other,1
4,4,VizWiz_train_00000006.jpg,What is in this bottle?,"[{'answer_confidence': 'yes', 'answer': 'antis...",other,1


In [7]:
list_of_questions = df1.question

In [8]:
list_of_questions

0              Can you tell me what is in this can please?
1        Is this enchilada sauce or is this tomatoes?  ...
2                  What is the captcha on this screenshot?
3                                       What is this item?
4                                  What is in this bottle?
                               ...                        
13103                                Is there a past case?
13104                     Is there a progress bar visible?
13105                    What's the color for this laptop?
13106    (inaudible) can you see it? If so, then tell m...
13107                                         Is it a box?
Name: question, Length: 13108, dtype: object

In [9]:
list_ques = list_of_questions.to_list()

In [12]:
tokenized_word_in_questions = [word_tokenize(i) for i in list_of_questions]

In [13]:
flattened_tokens = sum(tokenized_word_in_questions, [])

In [14]:
filtered_tokens = [w for w in flattened_tokens if w.isalnum()] # removing punctuation and other symbols
filtered_tokens = [w.lower() for w in filtered_tokens]

In [15]:
unique_tokens = set(filtered_tokens)

In [16]:
len(unique_tokens)

2992

In [ ]:
#Running the BERT model

In [46]:
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True,
                                  )                             # load the model from the internet 
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased') # inbuilt tokenizer 

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [20]:
def bert_text_preparation(text, tokenizer):
    """ Takes a string and performs preprocessing - special tokens (cls and sep); tokenization (tokens to ids and tokens to segment ids)
    Preparing the input for BERT
        
    Returns:
        list: List of BERT-readable tokens
        tokens_tensor: Torch tensor with token ids [no_of_tokens]
        segments_tensor: Torch tensor segment ids [no_of_tokens]
    
    
    """
    marked_text = "[CLS] " + text + " [SEP]"
    tokenized_text = tokenizer.tokenize(marked_text) # tokenize the sentence, not always word wise separately, sometimes divides into subwords also. 
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)  # convert tokenized texts to ids for the model to recognize the words 
    segments_ids = [1]*len(indexed_tokens) # BERT expects sentence pairs as well, if two sentences [0, 1] assigned to first and second sentence resp. In our case, we have one sentence per question, therefore just [1]

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    return tokenized_text, tokens_tensor, segments_tensors

In [ ]:
def get_bert_embeddings(tokens_tensor, segments_tensors, model):
    """Get embeddings from an embedding model
    
    Returns:
        List of list of floats of size [no_of_tokens, no_of_embedding_dimensions]
        containing embeddings for each token
    
    """
    
    with torch.no_grad():                                   # Gradient calculation id disabled, no computation graph creaed as backprop won't be running here 
        outputs = model(tokens_tensor, segments_tensors) [layer no, batch_no, word/token no, hidden_unit]
        # Removing the first hidden state
        # The first state is the input state
        hidden_states = outputs[2][1:]

    
    token_embeddings = hidden_states[-1] # Getting embeddings from the final BERT layer, 12th layer 
    token_embeddings = torch.squeeze(token_embeddings, dim=0) # Collapsing the tensor into 1-dimension
    list_token_embeddings = [token_embed.tolist() for token_embed in token_embeddings] # Converting torchtensors to lists

    return list_token_embeddings

In [ ]:
target_word_embeddings = []

cnt = 0 
for text in list_ques:
    tokenized_text, tokens_tensor, segments_tensors = bert_text_preparation(text, tokenizer) # prepare text for BERT
    list_token_embeddings = get_bert_embeddings(tokens_tensor, segments_tensors, model) 
    # print(text)
    # print(cnt)
    # cnt += 1
    target_word_embeddings.append(list_token_embeddings)

Streaming output truncated to the last 5000 lines.
What is this
8369
What color is this please?
8370
Which part of the box have internal control?
8371
What is in this box?
8372
Is this a photo or a blank page?
8373
Kind of cereal?
8374
Okay, so what kind of card is this?
8375
What is in this can?
8376
Can you tell me what's in this box of frozen dinner?
8377
What's the content of this screen?
8378
What is this?
8379
What's in this box? Thank you.
8380
Can you tell which phone number this is if possible please?
8381
Hi there, I'm sending this photo along because I want to know what the captcha is.  If you could send that back I'd really appreciate it.  Thank you.
8382
What kind of beer is this please?
8383
What does this microwave say?
8384
What is this?
8385
What does this shirt look like?
8386
Is this garlic?
8387
What time does the display on the stove say?
8388
What is the name of this body spray?
8389
Can you tell me what kind of flower this is?
8390
Is this right side up?
8391
Wha

In [ ]:
df_target = pd.DataFrame(target_word_embeddings)

In [ ]:
df_target.to_csv("BERT_embeddings.csv")

In [33]:
import pickle

In [ ]:
with open('BERT_Embeddings.pkl', 'wb') as f:
  pickle.dump(target_word_embeddings, f)

In [ ]:
# sentence embedding

In [17]:
!pip install sentence-transformers

     |████████████████████████████████| 78 kB 3.6 MB/s 
     |████████████████████████████████| 1.2 MB 16.4 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=688330bd8ae30c255ffdea658f7354ace6f2b7ee1097cd0bf9f91c18ca1957f1
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers


In [18]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [31]:
sentence_embeddings = sbert_model.encode(list_ques[0])

In [32]:
sentence_embeddings.shape

(768,)

In [38]:
target_sentence_embeddings = []
cnt = 0
for text in list_ques:
    sentence_embeddings = sbert_model.encode(text)
    # print(text)
    # print(cnt)
    # cnt += 1
    target_sentence_embeddings.append(sentence_embeddings)

Streaming output truncated to the last 5000 lines.
Tell me what this is.
10608
Which product is this?
10609
What is in this Progesso whatever? I don't know what's in it.
10610
What kind of pop is this?
10611
What is on, what type of bottle is this? 
10612
What's in the bowl?
10613
What is this cheese?
10614
what is this?
10615
Is selected on the menu on the tv?
10616
What does this say?
10617
What is the product, in the bottle, in the picture?
10618
Sparks contain
10619
What is this?
10620
What is on the label of this bottle?
10621
Says this is a knob on a dryer and it's in the shape of a block and it probably has like normal or delicate press things.
10622
Can you tell me what these ones say?
10623
What is this?
10624
What is the name of this CD and the disc number?
10625
Is it tuna with sweet corn or tune without sweet corn
10626
Which is this?
10627
What is this item?
10628
What kind of canned food is this?
10629
What is that?
10630
what is this set to?
10631
What color is this jump

In [42]:
with open('BERT_sentence_Embeddings.pkl', 'wb') as f:
  pickle.dump(target_sentence_embeddings, f)